# Assignment 2 - Epipolar Geometry and 3D Reconstruction 

First name: Marcel
<br>
Last name: Zauder
<br>
Matriculation number: 16-124-836

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

import numpy as np
from PIL import Image
import ipyvolume as ipv
import matplotlib.pyplot as plt

from utils import decompose_essential_matrix, infer_3d, ransac

## Part II: 3D Model Reconstruction

### Load matched points
We provide a synthetic pair of images where noisy correspondences are known.

In [ ]:
left = np.array(Image.open(os.path.join('MatchedPoints','left.jpg')), dtype=np.float32).mean(2)/255
right = np.array(Image.open(os.path.join('MatchedPoints','right.jpg')), dtype=np.float32).mean(2)/255

In [ ]:
plt.subplot(1,2,1)
plt.imshow(left, cmap='gray')
plt.title('Left image')
plt.subplot(1,2,2)
plt.imshow(right, cmap='gray')
plt.title('Right image')

In [ ]:
ilias_username = "z.m.matti" # TODO
A = np.loadtxt(f'MatchedPoints/Matched_Points_{ilias_username}.txt')

In [ ]:
M, N = A.shape
leftPoints = np.concatenate((A[:,2:4].T, np.ones((1, M))), axis=0)
rightPoints = np.concatenate((A[:,0:2].T, np.ones((1, M))), axis=0)

### Calibration matrix and focal length from the given file

In [ ]:
fl = 4
K = np.array([
    [-83.33333, 0.00000, 250.00000],
    [0.00000, -83.33333, 250.00000],
    [0.00000, 0.00000,   1.00000],
])

I = K.copy()

I[0,0] *= fl
I[1,1] *= fl

### Estimate Essential matrix E from F with RANSAC

In [ ]:
good_threshold = 0.001 # TODO
F, inliers = ransac(leftPoints, rightPoints, good_threshold)
print('Num outliers', leftPoints.shape[1] - inliers.sum(), ', outlier indices', np.argwhere(inliers == 0))
assert np.linalg.matrix_rank(F) == 2
print('Estimated fundamental matrix: ')
print(F)

# TODO: Estimate essential matrix E from F
E = I.T @ F @ I

print('Estimated essential matrix: ')
print(E)

### Compute rotation and translation between views

In [ ]:
# TODO: Compute rotation and translation between views. Complete decompose E
Il = np.linalg.solve(I, leftPoints)
Ir = np.linalg.solve(I, rightPoints)

Pl, Pr = decompose_essential_matrix(E, Il, Ir)

print('Estimated translation: ')
print(Pr[:,3])
print('Estimated rotation: ')
print(Pr[:,:3])

### Estimate the 3D points

In [ ]:
x3D = infer_3d(Il, Ir, Pl, Pr)
ipv.quickscatter(x=x3D[0,:], y=x3D[1,:], z=x3D[2,:])

In [ ]:
# Calculate homogenous transfomration matrices for both cameras from extrinsic cood
# Rotation of the Right Camera
Rr = np.array([[0.92848,  -0.12930,   0.34815],
               [0.00000,   0.93744,   0.34815],
               [-0.37139,  -0.32325,   0.87039]])

# Translation of the Right Camera
tr = np.array([-2.0, 2.0, 5.0])

# Compute homogenous transformation matrix of Right Camera

# Rotation of the Left Camera with respect to world frame
Rl = np.array([[1.00000,   0.00000,   0.00000],
               [0.00000,   0.92848,   0.37139],
               [0.00000,  -0.37139,   0.92848]])

# Translation of the Left Camera with respect to world frame
tl = np.array([0.0, 2.0, 5.0])